# Event Processing Analysis

### What's the goal?
Understand where Kalium is spending more time during event processing, in order to find possible performance improvements.


### Current context
- We've added [logs that measure how much time we're taking to process individual events](https://github.com/wireapp/kalium/pull/2827).
- The average time to process events oscilates every day between 368ms all the way up to 1280ms;
- 97,3% of the time spent processing events was processing end-to-end encrypted messages;
- 51% of the time spent processing messages was in order to process Text messages, even though they only account for 24,04% of the messages;
- `LastRead` is another interesting one: 18,57% of the count, but takes 24,3% of the time

## What even is measured within "Event Processing"?
We measure all the time taken from the beginning of `EventReceiver<EventType>.onEvent` until the event is handled completely.
For some Event Types, this can be super quick, like handling in-conversation Typing indication. Typing indication consists of just emitting to an in-memory flow and the event is fully processed.
For some events, this can be tricky. All End-To-End messages require parsing the encrypted blob, parsing the Protobuf, and then "actually" handling the different message types. Different message types often need completely different logic, so they can be as diverse as different event types.

## Starting plan of action
Collect data locally, by adding more logs and generating events.

My first fail was using an emulator. Running the debug of the Android app on the emulator resulted in nothing but weird looks. DataDog was giving hundreds of milliseconds as the average for event processing, while my emulator was taking about 20~30ms. 

How could that be? Running on an actual device – with the screen locked – made it right. An emulator has almost no apps installed competing for resources with the app being tested, and isn't prone to battery saving measures like CPU speed reduction, etc.
  
## Lesson #1
We can focus on "background performance": receive a push notification, sync, decrypt, show notifications.
Or on "foreground performance": open the app, sync, decrypt, show notifications and unread indicators.

The main difference is how the app might be subject to battery saving measures taken by the OS of a real device, and the time to perform the same actions might be completely different between the two scenarios.
Proposed action:
- Investigate further if we can add battery, back/fore-ground information, etc. to logs; This way we can track performance indication separately.

## Keep digging 

Considering how there are multiple steps to "event processing", I've added logs to gather more information about these steps and their "substeps".
Also, considering 97,3% of the time is spent during Messages processing, I focused on these.

Time spent during message events processing can be seen defined below in `TimeTaken`. 
 

Just adding some dependencies for manipulating data and Kotlin's datetime

In [13]:
%use kandy
%use dataframe

In [14]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-datetime:0.6.1")

Defining a way of representing event processing measurements and how to import data from the `logs`:

In [15]:

import kotlin.time.Duration
import kotlin.time.Duration.Companion.milliseconds

data class ProteusDecryption(
    val checkingSession: Duration,
    val decrypting: Duration,
    val savingSession: Duration,
) {
    val total: Duration = checkingSession + decrypting + savingSession
}

data class ProcessingMessage(
    /**
     * Dealing with possibility of LegalHold
     */
    val checkingLegalhold: Duration,
    /**
     * Actually handling the content of the message,
     * if LastRead, marking as read in the DB,
     * if Text, inserting into the DB, emitting notifications,
     * ...
     */
    val handlingMessageContent: Duration,
    /**
     * Other actions we take after the message is handled, like enqueueing delivery receipt
     * and enqueueing message deletion if the message is SelfDeleting
     */
    val postHandling: Duration
) {
    val total: Duration = checkingLegalhold + handlingMessageContent + postHandling
}

data class TimeTaken(
    val messageType: String,
    val total: Duration,
    val proteus: ProteusDecryption,
    val processingMessage: ProcessingMessage
) {
    /**
     * Unknown time can be things like parsing the Protobuf, converting the blob from Base64, etc.
     * In general: it shouldn't be slow, and it's less likely we can speed it up.
     */
    val unknownTime = total - proteus.total - processingMessage.total
}

fun getDataFromLogs(logFileName: String): List<TimeTaken> {
    val file = File("logs/$logFileName")
    val groups = file.readText().split("""^$""".toRegex(RegexOption.MULTILINE))

    val allData = groups.map {
        it.trim().split("\n").filter { it.isNotBlank() }
    }.filter {
        it.isNotEmpty()
    }.map { lines ->
        val logLine = lines[0]
        val measurements = logLine.split(";").map { it.split(" ").last() }
        val proteusDuration = ProteusDecryption(
            Duration.parse(measurements[0].trim()),
            Duration.parse(measurements[1].trim()),
            Duration.parse(measurements[2].trim()),
        )

        val processingLine = lines[1]
        val processingMeasurements = processingLine.split(";").map { it.split(" ").last() }
        val processingDuration = ProcessingMessage(
            Duration.parse(processingMeasurements[0].trim()),
            Duration.parse(processingMeasurements[1].trim()),
            Duration.parse(processingMeasurements[2].trim()),
        )
        val mainInfo = lines[2].split(" ")
        val timeTaken = mainInfo[0].toLong().milliseconds
        val messageType = mainInfo[1]
        TimeTaken(messageType, timeTaken, proteusDuration, processingDuration)
    }
    return allData
}

Ok, so let's do a simple analysis of how much time is spent during Proteus, Message handling and other.

In [16]:
@file:DependsOn("org.jetbrains.kotlinx:kotlinx-datetime:0.6.1")

val initialData = getDataFromLogs("InitialData.txt")

val pairs =
    initialData.map { it.proteus.total.inWholeMicroseconds to "Proteus" } +
            initialData.map { it.processingMessage.total.inWholeMicroseconds to "Processing" } +
            initialData.map { it.unknownTime.inWholeMicroseconds to "Other" }

val value by column(pairs.map { it.first })
val stepName by column(pairs.map { it.second })
val dataSet = dataFrameOf(value, stepName)
dataSet.groupBy(stepName).aggregate {
    sum(value) into "total"
}.plot {
    pie {
        slice("total")
        fillColor("stepName")
    }
    layout {
        title = "Time spent unpacking and processing message events"
        style(Style.Void)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="MD3FwG"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Time spent unpacking and processing message events"
},
"mapping":{
},
"data":{
"total":[3753588.0,4459976.0,804380.0],
"stepName":["Proteus","Processing","Other"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"slice":"total",
"fill":"stepName"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("MD3FwG");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Time spent unpacking and processing message events 
 
 
 
 
 
 
 
 
 stepName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Proteus 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Processing 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Other

While Processing clearly is the slowest part, the Proteus part is still interesting. Because it affects _all_ message types.
Processing can be very different between messages and requires an investigation for _each_ message type to find bottlenecks and improvement opportunities.

With that  in mind, let's dive a bit deeper into Proteus. Let's split it into smaller chunks and see what we can improve there.

In [17]:
val initialProteusMeasurements = initialData.map { it.proteus }
val checkingSession by column(initialProteusMeasurements.map { it.checkingSession.inWholeMilliseconds })
val decrypting by column(initialProteusMeasurements.map { it.decrypting.inWholeMilliseconds })
val savingSession by column(initialProteusMeasurements.map { it.savingSession.inWholeMilliseconds })
val eventNumber by column(buildList { repeat(initialProteusMeasurements.size) { i -> add(i + 1)} })
val detailedFrame = dataFrameOf(eventNumber, checkingSession, decrypting, savingSession)
    .gather(checkingSession, decrypting, savingSession)
    .into("measurements", "timeSpent")

detailedFrame.plot {
    layout.title = "Time taken in each phase of Proteus Unpacking, for different events"
    points {
        x("measurements")
        y("timeSpent")
        color("measurements")
        position = Position.jitter()
    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="siCBkh"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Time taken in each phase of Proteus Unpacking, for different events"
},
"mapping":{
},
"data":{
"timeSpent":[8.0,63.0,30.0,10.0,36.0,21.0,19.0,25.0,49.0,10.0,30.0,50.0,40.0,77.0,21.0,5.0,20.0,46.0,9.0,20.0,20.0,37.0,117.0,34.0,129.0,61.0,12.0,9.0,69.0,19.0,34.0,106.0,57.0,16.0,10.0,76.0,7.0,85.0,56.0,11.0,60.0,13.0,6.0,13.0,85.0,13.0,28.0,30.0,6.0,12.0,14.0,18.0,56.0,10.0,5.0,22.0,43.0,22.0,15.0,18.0,7.0,11.0,25.0,37.0,50.0,31.0,6.0,10.0,17.0,7.0,77.0,60.0,6.0,76.0,12.0,9.0,17.0,10.0,7.0,10.0,8.0,14.0,28.0,92.0,18.0,14.0,35.0,9.0,10.0,32.0,58.0,15.0,46.0,48.0,82.0,15.0,4.0,34.0,17.0,27.0,28.0,18.0,27.0,66.0,30.0,48.0,90.0,57.0,44.0,53.0,49.0,12.0,35.0,9.0],
"measurements":["checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession","checkingSession","decrypting","savingSession"]
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"color",
"discrete":true
}],
"layers":[{
"mapping":{
"x":"measurements",
"y":"timeSpent",
"color":"measurements"
},
"stat":"identity",
"sampling":"none",
"position":"jitter",
"geom":"point",
"data":{
}
}]
};
 var plotContainer = document.getElementById("siCBkh");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 checkingSession 
 
 
 
 
 
 
 
 
 decrypting 
 
 
 
 
 
 
 
 
 savingSession 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 
 
 20 
 
 
 
 
 
 
 40 
 
 
 
 
 
 
 60 
 
 
 
 
 
 
 80 
 
 
 
 
 
 
 100 
 
 
 
 
 
 
 120 
 
 
 
 
 
 
 
 
 Time taken in each phase of Proteus Unpacking, for different events 


While the colours might cause seizure, it goes to show how random it is. In some times saving session is slow. In some times checking session is slow. In some times decrypting is slow.

One thing can definitely be seen from this, is that decrypting is the slowest part.

Unfortunately, decrypting is something we can't solve on our end. It's embedded within CoreCrypto / CryptoBox, and we are unable to change it.

Here's a simpler visualization:

In [18]:
val steps by columnOf("Checking Session", "Decrypting", "Saving Session")
val values by columnOf(checkingSession.sum(), decrypting.sum(), savingSession.sum())

val dataFrame = dataFrameOf(steps, values)
dataFrame.plot {
    pie {
        slice(values)
        fillColor(steps)
    }
    layout {
        title = "Time share in different Proteus phases"
        style(Style.Void)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="xIebLD"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Time share in different Proteus phases"
},
"mapping":{
},
"data":{
"values":[802.0,1631.0,1267.0],
"steps":["Checking Session","Decrypting","Saving Session"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"slice":"values",
"fill":"steps"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("xIebLD");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Time share in different Proteus phases 
 
 
 
 
 
 
 
 
 steps 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Checking Session 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Decrypting 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Saving Session

Clearly decrypting is the biggest offender. And as mentioned before, we can't do much about it.

However, we might have quick wins and reduce `Saving Session` and `Checking Session`, which account for almost 50% of the total Proteus time.

## Idea #1
Remove `Saving Session` completely. It's unnecessary as CoreCrypto will perform session saving in every `decrypt`. We're doing that unnecessarily.

## Idea #2
Cache existing sessions in-memory. Instead of using IO and relying solely on CoreCrypto to know if sessions exist or not, we can store known sessions in memory.
If sessions are deleted, we can remove from the memory cache. 

## Results
After doing the first idea, I collected data and stored it in `NotSavingSession.txt`. After that, I also did the second idea and collected data, bringing to the joined improvements.

In [19]:
val withoutSessionSavingData = getDataFromLogs("NotSavingSession.txt")
val withSessionCacheData = getDataFromLogs("WithExistingSessionCache.txt")

val initialTime = initialData.map { it.total.inWholeMilliseconds to "Initial" }
val withoutSessionSaving =
    withoutSessionSavingData.map { it.total.inWholeMilliseconds to "NoSessionSaving" }
val withCache = withSessionCacheData.map { it.total.inWholeMilliseconds to "WithSessionCache" }
val allData = (initialTime + withoutSessionSaving + withCache)
val timeTaken by column(allData.map { it.first })
val approach by column(allData.map { it.second })

val averages = listOf(initialTime.map { it.first }.average(), withoutSessionSaving.map { it.first }.average(), withCache.map { it.first }.average())
val dataFrame = dataFrameOf(timeTaken, approach)

dataFrame.plot {
    layout.title = "Average time taken to fully process events across different approaches"
    points {
        y(timeTaken)
        x(approach)
        color(approach)
        position = Position.jitter()
    }
    line {
        y(averages) {
            axis.name = "Time Taken (ms)"
        }
        x(approach.distinct()) {
            axis.name = "Approach"
        }
        color = Color.PURPLE
        this.tooltips(true)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="oVi0Gr"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Average time taken to fully process events across different approaches"
},
"mapping":{
},
"data":{
"timeTaken":[349.0,150.0,240.0,347.0,268.0,182.0,112.0,363.0,481.0,299.0,400.0,242.0,322.0,150.0,183.0,161.0,104.0,178.0,159.0,127.0,174.0,306.0,161.0,233.0,218.0,86.0,122.0,313.0,261.0,201.0,286.0,275.0,164.0,203.0,325.0,378.0,337.0,158.0,412.0,267.0,140.0,292.0,444.0,127.0,226.0,231.0,327.0,147.0,205.0,330.0,286.0,314.0,172.0,198.0,360.0,124.0,232.0,151.0,181.0,112.0,162.0,80.0,72.0,68.0,122.0,191.0,89.0,104.0,177.0,72.0,72.0,109.0,240.0,117.0,373.0,201.0,106.0,65.0,128.0,191.0,147.0,143.0,112.0,133.0,168.0,251.0,180.0,343.0,200.0,171.0,164.0,104.0,102.0,77.0,103.0,191.0,129.0,204.0,117.0,73.0,198.0,165.0,190.0,78.0,172.0,124.0,172.0,155.0,74.0,54.0,259.0,103.0,89.0,168.0,58.0,135.0,92.0,136.0,97.0,145.0,145.0,151.0,263.0,204.0,160.0,221.0,116.0,92.0,138.0],
"approach":["Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","Initial","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","NoSessionSaving","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache","WithSessionCache"]
},
"kind":"plot",
"scales":[{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"color",
"discrete":true
},{
"aesthetic":"y",
"name":"Time Taken (ms)",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true,
"name":"Approach"
}],
"layers":[{
"mapping":{
"y":"timeTaken",
"x":"approach",
"color":"approach"
},
"stat":"identity",
"sampling":"none",
"position":"jitter",
"geom":"point",
"data":{
}
},{
"mapping":{
"y":"y",
"x":"approach"
},
"stat":"identity",
"data":{
"y":[237.31578947368422,195.71794871794873,147.1153846153846],
"approach":["Initial","NoSessionSaving","WithSessionCache"]
},
"color":"

## First win
Clearly we're up to something. Processing events is clearly faster with the changes in Proteus. The average time went from 247ms, to 196ms without saving session, and then down to 147ms without saving session + caching existing sessions.

## But we're not over yet
Maybe we can also analyse the different parts of `ProcessingMessage` and identify what is taking more time? For that, we can join all the logs taken so far and ignore the Proteus part.

In [20]:
val allMeasurements = initialData + withoutSessionSavingData + withSessionCacheData
val allMessageProcessing = allMeasurements.map { 
    it.processingMessage
}
val checkingLegalhold by column(allMessageProcessing.map { it.checkingLegalhold.inWholeMilliseconds })
val handlingContent by column(allMessageProcessing.map { it.handlingMessageContent.inWholeMilliseconds })
val postHandling by column(allMessageProcessing.map { it.postHandling.inWholeMilliseconds })
val eventNumber by column(buildList { repeat(allMessageProcessing.size) { i -> add(i + 1)} })
val detailedFrame = dataFrameOf(eventNumber, checkingLegalhold, handlingContent, postHandling)
    .gather(checkingLegalhold, handlingContent, postHandling)
    .into("measurements", "timeSpent")

detailedFrame.plot { 
    points { 
        x("measurements")
        y("timeSpent")
        position = Position.jitter()
        color("measurements")
    }
}


<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="XLgv7H"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
"timeSpent":[5.0,130.0,0.0,23.0,52.0,0.0,12.0,100.0,0.0,86.0,141.0,0.0,9.0,106.0,0.0,3.0,99.0,0.0,4.0,41.0,0.0,11.0,142.0,0.0,131.0,124.0,0.0,12.0,144.0,8.0,6.0,179.0,0.0,17.0,106.0,0.0,19.0,112.0,0.0,8.0,50.0,0.0,21.0,52.0,0.0,4.0,62.0,0.0,5.0,40.0,0.0,5.0,79.0,0.0,3.0,78.0,0.0,4.0,49.0,0.0,4.0,119.0,0.0,18.0,139.0,0.0,12.0,84.0,0.0,26.0,55.0,0.0,7.0,104.0,0.0,5.0,30.0,0.0,3.0,87.0,0.0,4.0,106.0,0.0,28.0,144.0,0.0,4.0,105.0,0.0,39.0,101.0,0.0,6.0,72.0,0.0,14.0,86.0,0.0,24.0,92.0,0.0,3.0,191.0,0.0,13.0,161.0,0.0,33.0,122.0,0.0,16.0,79.0,0.0,0.0,97.0,0.0,10.0,181.0,0.0,9.0,75.0,0.0,5.0,138.0,0.0,4.0,346.0,0.0,14.0,43.0,0.0,5.0,104.0,0.0,33.0,58.0,0.0,4.0,89.0,0.0,9.0,87.0,0.0,4.0,82.0,0.0,10.0,139.0,0.0,6.0,125.0,0.0,28.0,121.0,0.0,9.0,101.0,0.0,4.0,111.0,0.0,5.0,210.0,0.0,7.0,26.0,0.0,43.0,84.0,0.0,5.0,65.0,0.0,6.0,57.0,0.0,5.0,37.0,0.0,4.0,61.0,0.0,4.0,20.0,0.0,9.0,16.0,0.0,6.0,14.0,0.0,10.0,25.0,0.0,7.0,45.0,0.0,4.0,29.0,0.0,4.0,16.0,0.0,40.0,24.0,0.0,6.0,21.0,0.0,19.0,11.0,0.0,4.0,58.0,0.0,4.0,120.0,0.0,4.0,52.0,0.0,79.0,47.0,0.0,10.0,106.0,0.0,25.0,16.0,0.0,0.0,30.0,0.0,8.0,87.0,0.0,43.0,84.0,0.0,22.0,82.0,0.0,18.0,39.0,0.0,12.0,59.0,0.0,4.0,43.0,0.0,4.0,74.0,0.0,17.0,160.0,0.0,29.0,32.0,0.0,8.0,110.0,0.0,26.0,100.0,0.0,32.0,77.0,0.0,13.0,59.0,0.0,22.0,54.0,0.0,7.0,56.0,0.0,4.0,30.0,0.0,13.0,23.0,0.0,5.0,63.0,0.0,7.0,45.0,0.0,4.0,82.0,0.0,5.0,41.0,0.0,5.0,23.0,0.0,5.0,84.0,0.0,6.0,44.0,0.0,23.0,73.0,0.0,15.0,19.0,0.0,7.0,95.0,0.0,30.0,31.0,0.0,19.0,70.0,0.0,9.0,19.0,0.0,4.0,18.0,0.0,4.0,30.0,0.0,13.0,114.0,0.0,28.0,16.0,0.0,4.0,14.0,0.0,36.0,41.0,0.0,5.0,32.0,0.0,4.0,38.0,0.0,4.0,47.0,0.0,4.0,26.0,0.0,14.0,46.0,3.0,5.0,76.0,0.0,19.0,76.0,0.0,5.0,56.0,0.0,11.0,99.0,0.0,13.0,78.0,0.0,31.0,24.0,0.0,50.0,69.0,0.0,5.0,11.0,0.0,9.0,45.0,0.0,6.0,83.0,0.0],
"measurements":["checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHandling","checkingLegalhold","handlingContent","postHa

## Bad news

While checking legalhold is definitely a bit more random (probably due to IO operations), handling each different content is definitely the biggest offender.
Here's a comparison across all different steps using the most improved data.

In [21]:
import org.jetbrains.kotlinx.kandy.ir.feature.FeatureName

val pairs =
    withSessionCacheData.map { it.proteus.checkingSession.inWholeMicroseconds to "Checking Session" } +
            withSessionCacheData.map { it.proteus.decrypting.inWholeMicroseconds to "Decrypting" } +
            withSessionCacheData.map { it.proteus.savingSession.inWholeMicroseconds to "Saving Session" } +
            withSessionCacheData.map { it.processingMessage.checkingLegalhold.inWholeMicroseconds to "Checking Legalhold" } +
            withSessionCacheData.map { it.processingMessage.handlingMessageContent.inWholeMicroseconds to "Handling Content" } +
            withSessionCacheData.map { it.processingMessage.postHandling.inWholeMicroseconds to "Post Handling" } +
            withSessionCacheData.map { it.unknownTime.inWholeMicroseconds to "Other" }

val value by column(pairs.map { it.first })
val stepName by column(pairs.map { it.second })
val dataSet = dataFrameOf(value, stepName)
dataSet.groupBy(stepName).aggregate {
    sum(value) into "total"
}.plot {
    pie {
        slice("total")
        fillColor("stepName")
    }
    layout {
        title = "Time spent unpacking and processing message events after improvements"
        style(Style.Void)
    }
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.3.3/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="qfRmXn"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"ggtitle":{
"text":"Time spent unpacking and processing message events after improvements"
},
"mapping":{
},
"data":{
"total":[6634.0,2872468.0,232.0,721930.0,2952363.0,6072.0,1090122.0],
"stepName":["Checking Session","Decrypting","Saving Session","Checking Legalhold","Handling Content","Post Handling","Other"]
},
"kind":"plot",
"scales":[{
"aesthetic":"fill",
"discrete":true
}],
"layers":[{
"mapping":{
"slice":"total",
"fill":"stepName"
},
"stat":"identity",
"sampling":"none",
"position":"identity",
"geom":"pie",
"data":{
}
}],
"theme":{
"name":"classic",
"axis":{
"blank":true
},
"line":{
"blank":true
},
"axis_ontop":false,
"axis_ontop_y":false,
"axis_ontop_x":false
}
};
 var plotContainer = document.getElementById("qfRmXn");
 LetsPlot.buildPlotFromProcessedSpecs(plotSpec, -1, -1, plotContainer);
 </script>
 </body>
</html>"> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Time spent unpacking and processing message events after improvements 
 
 
 
 
 
 
 
 
 stepName 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Checking Session 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Decrypting 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Saving Session 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Checking Legalhold 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Handling Content 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Post Handling 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Other

## What's next?

### CoreCrypto transactions
CoreCrypto has plans to create a transaction API so we can batch-decrypt messages, which should speed up the `decrypting` part.

### Check "Other" slice
Perhaps there is still a bit of time to gain in the "Other" part of the code, which is related to Protobuf parsing, Base64 decoding, etc. ?

### Dive deeper into each message type
Keep in mind that most of the events collected for the experiments above were Text events. Which seems to be one of the worst performing messages we process.
This is definitely the most demanding job, as it requires pretty much one investigation for Text, one for LastRead, one for Asset, one for Receipt, etc.
But might be one of the most fruitful ones, and strikes a good balance between a major refactor that would be with CoreCrypto transactions, and small / segmented improvements across the processing pipeline.